In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#  Telecom Dataset for Customer Churn

1. Dataset Description ¶

The dataset provided is from a telecom company which has a record of customer information and their churn.
Our goal is to predict the customers who might stop using their services.
It's a binary classification problem since we have our target column, the churn variable with values 0(No) or 1(yes)

Column Description :

    Customer ID : Unique ID of customer
    gender : Two categories Male and Female
    Senior Citizen : Two categories 0 or 1
    Partner : Yes or No
    Dependents : Yes or No
    tenure : How long have they been with the company
    Phone Service : Yes or No
    Multiple Lines : Yes, No or No phone service
    Internet Service : DSL, Fiber Optics or No
    Online Security : Yes, No or No internet service
    Online Backup : Yes, No or No internet service
    Device Protection : Yes, No or No internet service
    Tech Support : Yes, No or No internet service
    Streaming TV : Yes, No or No internet service
    Streaming Movies : Yes, No or No internet service
    Contract : Month-to-month, One year, Two year
    Paperless Biling : Yes or No
    Payment Method : Electronic check, Mailed check, Bank Transfer(automatic), Credit Card(automatic)
    Monthly Charges : Numeric value
    Total Charges : Numeric value
    Churn : Yes or No



In [ ]:
#importation of data manipulation, plotting and grid formating Modules
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

# Importing Classifier Modules
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

#metrics evaluation Modules
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import roc_auc_score

In [ ]:
from matplotlib import rcParams

# figure size in inches
from IPython.core.pylabtools import figsize

In [ ]:
pd.set_option('display.max_columns',21)
Teldata = pd.read_csv('/kaggle/input/telcom-customer-churn/Telco-Customer-Churn.csv')

# 1.DATA PREPARATION AND FORMATTING

In [ ]:
Teldata.shape

In [ ]:
Teldata.head()#showing a snapshot of the data

In [ ]:
Teldata.info()

**From the data, we have 21 columns among those; we have 3 numerical columns and 18 categorical columns.
We can also see that there are no variables with missing data.
Lets reformat the totalcharges variable from categorical to numerical variable.**

In [ ]:
Teldata['TotalCharges']=pd.to_numeric(Teldata['TotalCharges'],errors='coerce')

In [ ]:
#convertion to numeric type creates some null value in this column.
Teldata['TotalCharges'].isnull().sum() 

In [ ]:
#lets fill in the null value with the median value of the column
Teldata['TotalCharges']= Teldata['TotalCharges'].fillna(Teldata['TotalCharges'].median())

In [ ]:
#The nan values are now filled.
Teldata['TotalCharges'].isnull().sum()

In [ ]:
#brief summary on the numerical columns
Teldata.describe()

**We have four numerical type columns displayed above with maximum tenure of 72 years of a customer,
Maximum monthly charge being 118 with a mean of 18.**

In [ ]:
#preprocessing on the target variable
Teldata['Churn']=Teldata['Churn'].map({'Yes':'lost_Customers','No':'present_Customers'})

In [ ]:
Teldata['Churn'].value_counts()

In [ ]:
churn_data=Teldata['Churn'].value_counts()
churn_data
piechart=plt.pie(x=churn_data,labels=churn_data.keys(),autopct="%.1f%%")

In [ ]:
# Let's plot the Churn class against the Frequency
fig = plt.figure(figsize=(5,8))

labels = ['present_customers','lost_customers']
classes = pd.value_counts(Teldata['Churn'], sort = True)
classes.plot(kind = 'bar',rot=0)
plt.title("Churn class distribution")
plt.xticks(range(2), labels)
plt.xlabel("Classes")
plt.ylabel("Frequency")

**Considering the plots above(pie plot and bar plot)
There is some class imbalance in the churn variable where the lost customers are about 1/4 of the present customers ; There are 73.5% present customers  and 26.5% lost customers .
The imbalance in the dataset may lead to predictors making false positives/negatives because the churn data is undersampled.The ratio of existing customers to lost customers is not proportionate. We will deal with this issue during the modelling phase**

# **2.EXPLANATORY DATA ANALYSIS**

In [ ]:
#Lets looks at the relationship between Churn Variable values and the Tenure
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(Teldata[Teldata["Churn"]=='lost_Customers']["tenure"], bins=15, alpha=0.8, color="blue", label="lost customers")
ax.hist(Teldata[Teldata["Churn"]=='present_Customers']["tenure"], bins=15, alpha=0.8, color="green", label="present customers")

ax.set_xlabel("Tenure")
ax.set_ylabel("Count of customers")

fig.suptitle("Tenure vs. Churn for Telco customers")

ax.legend();

**Generally the highest number of lost customers had a lower tenure of between 0-5. The histogram becomes skewed with increasing tenure with the least number of lost customers having a tenure of between 55 to 70.
Generally, we had more present customers with a tenure between 0 and 5, 
The number of present customers gradualy decreases with increasing tenure and reverses between 50-70 where we see an increase in the number of present customers with most of them having a tenure of between 65-70..**

In [ ]:
#Lets look at the relationship between the values of the Churn Variable and the Monthly Charges
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(Teldata[Teldata["Churn"]=='lost_Customers']["MonthlyCharges"], bins=15, alpha=0.8, color="blue", label="lost customers")
ax.hist(Teldata[Teldata["Churn"]=='present_Customers']["MonthlyCharges"], bins=15, alpha=0.8, color="green", label="present customers")

ax.set_xlabel("MonthlyCharges ")
ax.set_ylabel("Count of customers")

fig.suptitle("MonthlyCharges  vs. Churn for Telco customers")

ax.legend();

**The histogram becomes fairly skewed with increasing Monthly Charges with the highest number of present customers having a monthy charge of between 20 and 25.
Generally, there are few lost customers whose monthly charges ranges from 25-65.
The highest number of lost customers had a monthly charge of between 65-110**

In [ ]:
#Lets look at the relationship between Total charges and the individual values of the Churn Variable.
fig, ax = plt.subplots(figsize=(10,8))

ax.hist(Teldata[Teldata["Churn"]=='lost_Customers']["TotalCharges"], bins=15, alpha=0.8, color="blue", label="lost customers")
ax.hist(Teldata[Teldata["Churn"]=='present_Customers']["TotalCharges"], bins=15, alpha=0.8, color="green", label="present customers")

ax.set_xlabel("TotalCharges ")
ax.set_ylabel("Count of customers")

fig.suptitle("TotalCharges  vs. Churn for Telco customers")

ax.legend();

**Considering the histogram above, the highest number of lost customers had a total monthly charge of between 0-500, the number of lost customers becomes highly skewed with increase in the total charges.
We can also see that the highest number of present customers have a total charge of between 0-500 and decreases to between 500-2000, the numbers continue to gradualy decrease with increase in the total charge..**

**Since the variable seniorcitizen has two possible values 1 or 0 , True or False, lets set it's 
data type to a categorical type.**

In [ ]:
Teldata['SeniorCitizen']=Teldata['SeniorCitizen'].astype('O')

In [ ]:
#brief summary on the numerical columns
Teldata.describe(include=['O'])

In [ ]:
Teldata['SeniorCitizen'].value_counts()

**The code above shows us that the senior citizens are about 1/4 of all the customers who have used the service.**

In [ ]:
Teldata.groupby(['SeniorCitizen','Churn']).customerID.count()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="SeniorCitizen", data=Teldata,ax=ax)
ax.legend(title="SeniorCitizen")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("SeniorCitizen vs. Churn Variable categories");

**We can see that from the code above, among those using the service,the Seniorcitizen are about 1/4 of them.
Among those who have left the Service,the SeniorCitizens are also about 1/4 of them as well**

In [ ]:
#Code below shows that we had more almost same number of males and females in this dataframe
Teldata['gender'].value_counts()

In [ ]:
Teldata.groupby(['gender','Churn']).customerID.count()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="gender", data=Teldata,ax=ax)
ax.legend(title="gender")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("Gender vs. Churn Variable categories");

**From the code above, we can see that there are slightly more males still using the service than female counterparts,and slightly more females who stop using the service than their male counterparts.**

In [ ]:
#Code below shows that we had alittle more customers without partners than those who had them.
Teldata['Partner'].value_counts()

In [ ]:
Teldata.groupby(['Partner','Churn']).customerID.count()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="Partner", data=Teldata,ax=ax)
ax.legend(title="Partner")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("Partner vs. Churn Variable categories");

**From the code above, we can see that we have slightly more customers with partners using the service than those without partners. We can also see that there are less people who have stopped using the service with partners than those without partners.
This implies that probably people with partners are more contented with the service than those without partners.**

In [ ]:
#Code below shows that customers with dependents are about half of those without dependents
Teldata['Dependents'].value_counts()

In [ ]:
Teldata.groupby(['Dependents','Churn']).customerID.count()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="Dependents", data=Teldata,ax=ax)
ax.legend(title="Dependent")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("Dependents vs. Churn Variable categories");

**From the code above, we can see that the number of people using the service without dependents is slightly more than twice the number of those using with dependents
However, we can also see that we have a lot more people without dependents who have stopped using the service than those with dependents who have also stopped using the service.**

In [ ]:
#Code below shows that customers not using the phone service are about 1/4 of those using it.
Teldata['PhoneService'].value_counts()

In [ ]:
Teldata.groupby(['PhoneService','Churn']).customerID.count()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="PhoneService", data=Teldata,ax=ax)
ax.legend(title="PhoneService")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("PhoneService vs. Churn Variable categories");

**From the code above, we have alot more people with phoneservice who are using the telcom service compared to those without phoneservice who also use the service.
However we also have alot more people who have stopped using the service with phoneservice as compared to their counterpart without phone service and they have also stopped using the service.**

In [ ]:
Teldata['MultipleLines'].value_counts()

**Since we are interested in a yes or no value of multiplelines, we can reformate No phone service to be in the same category of No multiple lines.**

In [ ]:
Teldata['MultipleLines']=Teldata['MultipleLines'].replace('No phone service','No')

In [ ]:
Teldata['MultipleLines'].value_counts()# new value counts with increased No values.

In [ ]:
Teldata.groupby(['MultipleLines','Churn']).customerID.count()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="MultipleLines", data=Teldata,ax=ax)
ax.legend(title="MultipleLines")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("MultipleLines vs. Churn Variable categories");

**From the code above, we have much more people without multiple lines who are using the service than thier counterparts with multiple lines and are using the telcom service.
However,when u look at those who have stopped using the service,the number of those with multiple lines and those without are nearly the same.**

In [ ]:
Teldata['InternetService'].value_counts()

In [ ]:
Teldata.groupby(['InternetService','Churn']).customerID.count()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="InternetService", data=Teldata,ax=ax)
ax.legend(title="InternetService")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("InternetService vs. Churn Variable categories");

**From the code above, we can see that the highest number of people using the telcom service comes from those using DSL and Fiber optic cables respectively then followed by those who do not use any internetService.
We can also see that the highest number of people who have stopped using the service were those who were using fibre optic cables then followed by DSL  as internet service providers. Those who were not using any internet service had the least number who had stopped using the service.**

In [ ]:
Teldata['OnlineSecurity'].value_counts()

**Since we are interested in Yes or No value of onlinesecurity , we can reformate No internet service to be in the same category as No since one with no internet service is most likely to have no online security.**

In [ ]:
Teldata['OnlineSecurity']=Teldata['OnlineSecurity'].replace('No internet service','No')

In [ ]:
Teldata['OnlineSecurity'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="OnlineSecurity", data=Teldata,ax=ax)
ax.legend(title="OnlineSecurity")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("OnlineSecuirty vs. Churn Variable categories");

**We can see that there are alot more people using the service who do not have online security.
It can be noted that the number of those using the service and have online security is about half of those using the telcom service without online security.
There is generally alot more people who have stopped using the service without online security.
It can also be noted that the number who haved stopped using the service who had online security were about 1/4 of those who had stopped the service without online security.**

In [ ]:
Teldata['OnlineBackup'].value_counts()

**Since we are interested in a yes or no value of onlineBackup , we can reformate No internet service to be in the same category as No since one with no internet service is most likely to have no online backup.**

In [ ]:
Teldata['OnlineBackup'].value_counts()

In [ ]:
Teldata['OnlineBackup']=Teldata['OnlineBackup'].replace('No internet service','No')

In [ ]:
Teldata['OnlineBackup'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="OnlineBackup", data=Teldata,ax=ax)
ax.legend(title="OnlineBackup")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("OnlineBackup vs. Churn Variable categories");

**The code above shows us that we have alot more people using the service without onlinebackup than those using the service with online backup.
It can also be noted that among those who stopped using the service, those with onlinebackup are close to half of those who stopped using the product without online backup.**

In [ ]:
Teldata['DeviceProtection'].value_counts()

**Since we are interested in a yes or no value of DeviceProtection , we can reformate No internet service to be in the same category as No assuming that device protection is over the internet and therefore one with no internet service will mostly likely not have device protection over the internet.**

In [ ]:
Teldata['DeviceProtection']=Teldata['DeviceProtection'].replace('No internet service','No')

In [ ]:
Teldata['DeviceProtection'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="DeviceProtection", data=Teldata,ax=ax)
ax.legend(title="DeviceProtection")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("DeviceProtection vs. Churn Variable categories");

**From the code above, we can see that we have alot more people using the service without device protection over the internet than those using the service with device protection.
Among those who stopped using the product,those with device protection are about half of those without device protection.**

In [ ]:
Teldata['TechSupport'].value_counts()

**Considering that the specified techsupport is offered online, we can reformate No internet support to the No category since a client without internet service will not have the online techsupport.** 

In [ ]:
Teldata['TechSupport']=Teldata['TechSupport'].replace('No internet service','No')

In [ ]:
Teldata['TechSupport'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="TechSupport", data=Teldata,ax=ax)
ax.legend(title="TechSupport")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("TechSupport vs. Churn Variable categories");

**From the code above, we can see that we have alot more people using the service without TechSupport over the internet than those using the service with TechSupport.
Among those who stopped using the Service,those with TechSupport are about 1/4 of those without TechSupport.**

In [ ]:
Teldata['StreamingTV'].value_counts()

**Considering that streamingtv service is faciliated with the availability of an internet connection,
we can reformate No internet service to the No category since a client without internet service will not be able to have tv streaming.**

In [ ]:
Teldata['StreamingTV']=Teldata['StreamingTV'].replace('No internet service','No')

In [ ]:
Teldata['StreamingTV'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="StreamingTV", data=Teldata,ax=ax)
ax.legend(title="StreamingTV")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("StreamingTV vs. Churn Variable categories");

**From the code above, we can see that we have alot more people using the service without TV streaming over the internet than those using the service with TV Streaming.
Among those who stopped using the Service,those who are able to have TV streaming are about 3/4 of those who are not able to have TV Streaming.**

In [ ]:
Teldata['StreamingMovies'].value_counts()

**Considering that streamingMovies service is faciliated with the availability of an internet connection,
we can reformate No internet service to the No category since a client without internet service will not be able to have movie streaming.**

In [ ]:
Teldata['StreamingMovies']=Teldata['StreamingMovies'].replace('No internet service','No')

In [ ]:
Teldata['StreamingMovies'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="StreamingMovies", data=Teldata,ax=ax)
ax.legend(title="StreamingMovies")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("Streaming Movies vs. Churn Variable categories");

**From the code above, we can see that we have alot more people using the service without movie streaming over the internet than those using the service with movie Streaming over the internet
Among those who stopped using the Service,those who are able to have Movie streaming are about 3/4 of those who are not able to have movie Streaming.**

In [ ]:
Teldata['Contract'].value_counts()

**The code above shows us that the highest number of clients have a month to month contract with the telco company**

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="Contract", data=Teldata,ax=ax)
ax.legend(title="Contract")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("Contract vs. Churn Variable categories");

**The code above shows us that the highest number of clients using the service have a month to month followed by two year and lastly one year contract
Among those who have stopped using the service, month to month has the highest number, followed by one year and finally two year contract.**

In [ ]:
Teldata['PaperlessBilling'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(x="Churn", hue="PaperlessBilling", data=Teldata,ax=ax)
ax.legend(title="PaperlessBilling")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("PaperlessBilling vs. Churn Variable categories");

**From the code above, we can see that the highest number of people using the service prefer to use Paperless billing, those who do not use paperless billing and are also using the service are about 3/4 of those who use paperless billing.
However, among those who stopped using the service,those who were not using paperless billing were about 1/4 of those who were using paperless billing.**

In [ ]:
Teldata['PaymentMethod'].value_counts()

In [ ]:
Teldata.groupby(['PaymentMethod','Churn']).customerID.count()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.countplot(x="Churn", hue="PaymentMethod", data=Teldata,ax=ax)
ax.legend(title="PaymentMethod")
ax.set_xticklabels(["Using_Service", "Not_Using_Service"])
ax.set_xlabel("Churn")
fig.suptitle("PaymentMethod vs. Churn Variable categories");

**From the code above, we can see that the prefered method of payment for those using the service is mailed check although the difference from the other payment methods is not so significant.
However among those who stopped using the service, the highest number of people who left the service had previously used the electronic check paymethod, then followed by mailed check,bank transfer and lastly credit card.**

# 3. CORRELATION OF FEATURES

**Let's investigate the correlation of  numerical features in relation to the target.**

In [ ]:
#select the numerical columns
num_data=Teldata.loc[:,Teldata.dtypes!=np.object]

In [ ]:
#Preprocess the target column to fit in the numerical data for correlation purposes
num_data['Churn']=Teldata.loc[:,'Churn']
ohot=pd.get_dummies(num_data['Churn'])

In [ ]:
num_data=num_data.drop(['Churn'],axis=1)
num_data=num_data.join(ohot)

In [ ]:
# heat map of correlation of features
correlation_matrix = num_data.corr()
fig = plt.figure(figsize=(8,4))
sns.heatmap(correlation_matrix,vmax=0.8,annot=True) 
plt.show()

**From the correlation matrix,the correlation coefficient of lost and present customers to all feature columns are identical in numbers, with mirroring signs (+ and -). 
We will use all  of the numerical columns here in our modelling process since the features happen to have a considerable good correlation to the churn variable.**

**We shall also use all the categorical features since our exploratory data analysis showed that each of the features has a significant effect on the churn variable**.

# 4 FEATURE ENGINEERING

In [ ]:
#We will drop the customerId columns from the Teldata dataframe.
#Do some preprocessing on the Churn variable
Teldata['Churn'] = Teldata['Churn'].map({'present_Customers': 0, 'lost_Customers': 1})
train = Teldata.drop(columns=['customerID'],axis=1) 

In [ ]:
Train_df_OH= pd.get_dummies(train,drop_first=True)

In [ ]:
print(Train_df_OH.columns)

In [ ]:
Train_df_OH.head()

In [ ]:
Train_df_OH.shape

In [ ]:
target = Train_df_OH['Churn']
train = Train_df_OH.drop(columns='Churn',axis=1)

# 5. MODELLING

# Since there is an imbalance within the Churn target variable, we are going to build two models where,
# 1. We build our model with  the imbalanced churn variable
# 2. We build our model with a balanced churn variable  using SMOTE

**1. Model with imbalanced churn Variable.**

In [ ]:
#stratified sampling ensures the even distribution of classes of the target variable.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train,target, test_size=0.2, random_state=1,
                                                    stratify=target)

In [ ]:
X_train.shape , y_train.shape, X_test.shape,y_test.shape

In [ ]:
#Function to train and evaulate a given model
def fit_Evaulate(model,X_train,y_train):
    
    model.fit(X_train, y_train)
    y_pred=model.predict_proba(X_test)[:,1]#predict probabilities for positive class
    auc = roc_auc_score(y_test, y_pred)
    score=model.score(X_train,y_train)*100
    
    return auc,score   

# Model evaluations on various models.

1. LOGISTIC REGRESSION

2. SUPPORT VECTOR CLASSIFIER

3. K-NEAREST NEIGHBOUR CLASSIFIER

4. RANDOM FOREST CLASSIFIER

5. GRADIENT BOOSTING CLASSIFIER

In [ ]:
Log_Reg=LogisticRegression()
lOGR_auc,LOG_R_Score = fit_Evaulate(Log_Reg,X_train,y_train)
print("The Logistic Regression train Accuracy = %.2f"%(LOG_R_Score))
print('Logistic ROC AUC for test data: ROC AUC=%.3f' % (lOGR_auc))

In [ ]:
SVC_clf =SVC(probability=True)
SVC_auc,SVC_Score = fit_Evaulate(SVC_clf,X_train,y_train)
print("The SVC train Accuracy = %.2f"%(SVC_Score))
print('SVC ROC AUC for test data: ROC AUC=%.3f' %(SVC_auc))

In [ ]:
KNN_clf = KNeighborsClassifier(n_neighbors = 3)
KNN_auc,KNN_Score = fit_Evaulate(KNN_clf,X_train,y_train)
print("The KNeighbors Classifier train Accuracy = %.2f"%(KNN_Score))
print('KNN ROC AUC for test data: ROC AUC=%.2f' %(KNN_auc))

In [ ]:
RF_clf = RandomForestClassifier(n_estimators=100,class_weight="balanced")
RF_auc,RF_Score = fit_Evaulate(RF_clf,X_train,y_train)
print("The Random Forest train Accuracy = %.2f"%(RF_Score))
print('Random Forest ROC AUC for test data: ROC AUC=%.2f' %(RF_auc))

In [ ]:
GB_clf = GradientBoostingClassifier(n_estimators=100)
GB_auc,GB_Score = fit_Evaulate(GB_clf,X_train,y_train)
print("Gradient Boosting Classifier train Accuracy = %.2f"%(GB_Score))
print('Gradient Boosting ROC AUC for test data: ROC AUC=%.2f' %(GB_auc))

# Visualise the Train accuracy of the models.

In [ ]:
figsize(12,10)
fig = plt.figure()

# Dataframe to hold the results
model_comparison = pd.DataFrame({'model': ['LR_clf', 'SVC_clf',
                                           'KNN_clf', 'RF_clf',
                                            'GB_clf'],
                                 'Accuracy': [LOG_R_Score,SVC_Score,KNN_Score,RF_Score,GB_Score ]})

# Horizontal bar chart of train accuracy
model_comparison.sort_values('Accuracy').plot(x = 'model', y = 'Accuracy', 
                                kind = 'barh',color = 'Green', edgecolor = 'black')
# Plot formatting
plt.ylabel('Model'); plt.yticks(size = 10); plt.xlabel('Accuracy Score'); plt.xticks(size = 10)
plt.title('Model Comparison on Train Accuracy Score', size = 14);

**From the bar graph above, we can see that the Random forest classifier has the best accuracy performance on the train data. However since the data  has a class imbalance in the customer churn variable, we shall use the roc_auc metric to select the best model for evaluation performance**.

**We are going to use a bar graph below to visualise the model that produced the highest roc_auc,
which will become the model that we shall tune for our classification problem.**

In [ ]:
figsize=(12,10)
fig = plt.figure()

# Dataframe to hold ROC_AUC results
ROC_AUC_comparison = pd.DataFrame({'model': ['LogReg', 'SVC',
                                           'KNN', 'RF',
                                            'GB'],
                                 'Roc_Auc': [lOGR_auc,SVC_auc,KNN_auc,RF_auc,GB_auc ]})

# Horizontal bar chart of Roc_auc values
ROC_AUC_comparison.sort_values('Roc_Auc').plot(x = 'model', y = 'Roc_Auc', 
                                kind = 'barh',color = 'Blue', edgecolor = 'black')
# Plot formatting
plt.ylabel('Model'); plt.yticks(size = 10); plt.xlabel('Roc AUC Model Values'); plt.xticks(size = 10)
plt.title('Model Comparison for Various Roc Auc values', size = 14);
plt.show()

**The best performing models are Gradient Boosting classifiers and logistic regression**

Lets consider the two possible models, Logistic regression and gradient boosting and furthure tune them to find out which one performs better.
**We shall use grid search to find optimum parameters**

In [ ]:
LR_model= LogisticRegression()

grid_parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] , 'penalty':['l1','l2'] }

In [ ]:
from sklearn.model_selection import GridSearchCV

LR_grid= GridSearchCV(LR_model,grid_parameters,cv=10)

In [ ]:
LR_grid.fit(X_train,y_train)

In [ ]:
print(LR_grid.best_params_)

In [ ]:
y_pred_ProbLOGLR=LR_grid.predict_proba(X_test)[:,1]#predict probabilities for positive class
lr_auc_grid = roc_auc_score(y_test, y_pred_ProbLOGLR)
LOG_R_score=LR_grid.score(X_train,y_train)*100
print("The Logistic Regression grid train Accuracy = %.2f"%(LOG_R_score))
print('Logistic ROC AUC for test data on grid search: ROC AUC=%.3f' % (lr_auc_grid))

**The grid perform obtained from tuning logistic regression does not defer from the default model performance**

**Lets now tune the gradient boosting classifier**

In [ ]:
model = GradientBoostingClassifier(random_state=1)

In [ ]:
# parameter grid

parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.5, 0.1, 0.2],
    "min_samples_leaf": [2,3,4,6],
    "max_depth":[3,5,7,10,15],
    "max_features":["log2","sqrt"],
     'subsample': [0.6,0.7,0.8],
    "n_estimators":[300,400,500,800,1000]
     }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=model,
                               param_distributions=parameters,
                               scoring='roc_auc',
                               cv=10, n_iter=30, 
                               n_jobs = -1, verbose = 1, 
                               random_state=42)

In [ ]:
random_cv.fit(X_train,y_train)

In [ ]:
random_cv.best_params_

In [ ]:
random_cv.best_score_

In [ ]:
#Retrainig our model with grid search to optimize it's performance by varying the estimaators
# Create a range of trees to evaluate
# max_features ='sqrt',
param_grid = {'n_estimators': [200,300,400,500 ],"max_features":["log2","sqrt"]}
model =  GradientBoostingClassifier(max_depth=3,
                                   loss = 'deviance',
                                   subsample=0.6,
                                   learning_rate=0.01,
                                   min_samples_leaf = 6,
                                  random_state = 4)

In [ ]:
grid_clf =GridSearchCV(GradientBoostingClassifier(random_state=1), 
                   param_grid,scoring='roc_auc',cv=5, n_jobs=-1)

In [ ]:
grid_clf.fit(X_train,y_train)

In [ ]:
grid_clf.best_estimator_

In [ ]:
 grid_clf.best_score_

**We can note that the Gradient boosting classifier performed much better than Logistic Regression.
Tuning the gradient boosting classifier with Random search and grid search produced similar results.
We shall consider the final performance of grid search as the model to use to make our prediction.**

In [ ]:
final_model_GB=grid_clf.best_estimator_

In [ ]:
y_pred=final_model_GB.predict_proba(X_test)[:,1]#predict probabilities for positive class
#code below will threshold the probabilities to give class predictions
y_pred = np.where(y_pred > 0.5, 1,0)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
#comfusion matrix plot
Label_1 = ['Predicted_Not_Churn', 'Predicted_Churn']
Label_2 = [ 'True Not_Churn','True Churn', ]
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, xticklabels=Label_1, yticklabels=Label_2, annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('')
plt.xlabel('Predicted class')
plt.show()

**From the heatmap above, we can see that the model mis-classified 96 customers as False Positives(FP), implying that it classified them among customers who stopped using the service and yet they are still using the telcom service in the true sense.
The model also mis-classified 178 customers as False Negatives (FN), implying that it classified them among those customers who were using the service and yet in real sense they had stopped using the service**.

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(final_model_GB, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

# **We can see that Tenure, Monthly Charges and Total Charges  are the three most important features that this model considered in classifying which customers will churn or not.**

# 2. Building a Model with Balanced Churn Variable Values.

**Balancing the Training Dataset**

**The Churn customer takes only 26.7 % therefore a need to balance it.
To balance the dataset, we will utilize SMOTE. SMOTE (Synthetic Minority Oversampling Technique) consists of synthesizing elements for the minority class, based on those that already exist. It works by randomly picking a point from the minority class and computing the k-nearest neighbors for this point. The synthetic points are added between the chosen point and its neighbors.**

**First we split our training and test data. SMOTE will be applied only to training dataset so that the prediction will use original dataset with no synthetic datapoints**

In [ ]:
from imblearn.over_sampling import SMOTE
sm_object = SMOTE(sampling_strategy='minority', k_neighbors=9, random_state=2)
X_train_res, y_train_res = sm_object.fit_resample(X_train, y_train)

In [ ]:
#original train and y_train
X_train.shape, y_train.shape

In [ ]:
#orginal churn distribution
print(np.bincount(y_train))

In [ ]:
#New train shapes after transformation with added synthetic points.
X_train_res.shape, y_train_res.shape

In [ ]:
#This code below shows that the initial imbalance in the samples of the churn variable is removed.
#This create 4139 pos-tives and 4139 negatives in the churn variable.
# close to 3000 synthetic points are added to balance the churn values distribution.
print(np.bincount(y_train_res))

**We can now follow through with the same approach of training the models we looked at before to compare the results.**

**Lets train and evaulate the following models**

1.  LogisticRegression
2.  SVC Classifier
3.  K- Nearest Neighbour Classifier
4.  Random Forest Classifier
5.  Gradient Boosting Classifier**

In [ ]:
Log_Reg=LogisticRegression()
lr_auc2,LOG_R_Score2 = fit_Evaulate(Log_Reg,X_train_res,y_train_res)
print("The Logistic Regression train Accuracy = %.2f"%(LOG_R_Score2))
print('Logistic ROC AUC for test data: ROC AUC=%.3f' % (lr_auc2))

In [ ]:
SVC_clf =SVC(probability=True)
SVC_auc2,SVC_Score2 = fit_Evaulate(SVC_clf,X_train_res,y_train_res)
print("The SVC train Accuracy = %.2f"%(SVC_Score2))
print('SVC ROC AUC for test data: ROC AUC=%.3f' %(SVC_auc2))

In [ ]:
KNN_clf = KNeighborsClassifier(n_neighbors = 3)
KNN_auc2,KNN_Score2 = fit_Evaulate(KNN_clf,X_train_res,y_train_res)
print("The KNeighbors Classifier train Accuracy = %.2f"%(KNN_Score2))
print('KNN ROC AUC for test data: ROC AUC=%.2f' %(KNN_auc2))

In [ ]:
RF_clf = RandomForestClassifier(n_estimators=100,class_weight="balanced")
RF_auc2,RF_Score2 = fit_Evaulate(RF_clf,X_train_res,y_train_res)
print("The Random Forest train Accuracy = %.2f"%(RF_Score2))
print('Random Forest ROC AUC for test data: ROC AUC=%.2f' %(RF_auc2))

In [ ]:
GB_clf = GradientBoostingClassifier(n_estimators=100)
GB_auc2,GB_Score2 = fit_Evaulate(GB_clf,X_train_res,y_train_res)
print("Gradient Boosting Classifier train Accuracy = %.2f"%(GB_Score2))
print('Gradient Boosting ROC AUC for test data: ROC AUC=%.2f' %(GB_auc2))

We can also see from the code above even with the resampled churn variable, the random forest classifier has the best performance, however just as before, we shall use the roc_auc metric to determine the model with the best performance for this kind of data.

In [ ]:
figsize=(12,10)
fig = plt.figure()

# Dataframe to hold ROC_AUC results
ROC_AUC_comparison = pd.DataFrame({'model': ['LogReg', 'SVC',
                                           'KNN', 'RF',
                                            'GB'],
                                 'Roc_Auc': [lr_auc2,SVC_auc2,KNN_auc2,RF_auc2,GB_auc2 ]})

# Horizontal bar chart of Roc_auc values
ROC_AUC_comparison.sort_values('Roc_Auc').plot(x = 'model', y = 'Roc_Auc', 
                                kind = 'barh',color = 'Purple', edgecolor = 'black')
# Plot formatting
plt.ylabel('Model'); plt.yticks(size = 10); plt.xlabel('Roc AUC Model Values'); plt.xticks(size = 10)
plt.title('Model Comparison for Various Roc Auc values', size = 14);
plt.show()

**We can take note again that the Gradient boosting classifier is slightly a better classifier than Logistic Regression as well as when compared to the other classifiers.
We shall then Proceed to fine tune the Gradient boosting classfier to obtain an optimum performance.**

In [ ]:
model = GradientBoostingClassifier(random_state=2)

In [ ]:
# parameter grid

parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.5, 0.1, 0.2],
    "min_samples_leaf": [2,3,4,6],
    "max_depth":[3,5,7,10,15],
    "max_features":["log2","sqrt"],
     'subsample': [0.6,0.7,0.8],
    "n_estimators":[300,400,500,800,1000]
     }

In [ ]:
#we make use of the random search to get baseline parameters for the model.
from sklearn.model_selection import RandomizedSearchCV
random_cv2 = RandomizedSearchCV(estimator=model,
                               param_distributions=parameters,
                               scoring='roc_auc',
                               cv=10, n_iter=30, 
                               n_jobs = -1, verbose = 1, 
                               random_state=42)

In [ ]:
random_cv2.fit(X_train_res,y_train_res)

In [ ]:
random_cv2.best_params_

In [ ]:
random_cv2.best_score_

In [ ]:
#Retrainig our model with grid search to optimize it's performance by varying the estimaators
# Create a range of trees to evaluate
# max_features ='sqrt',
param_grid2 = {'n_estimators': [400,500,600,800,1000],"max_features":["log2","sqrt"]}
model =  GradientBoostingClassifier(max_depth=3,
                                   loss = 'deviance',
                                   subsample=0.8,
                                   learning_rate=0.1,
                                   min_samples_leaf = 4,
                                  random_state = 2)

In [ ]:
grid2_clf =GridSearchCV(GradientBoostingClassifier(random_state=1), 
                   param_grid2,scoring='roc_auc',cv=5, n_jobs=-1)

In [ ]:
grid2_clf.fit(X_train_res,y_train_res)

In [ ]:
grid2_clf.best_estimator_

In [ ]:
grid2_clf.best_score_

We can therefore see that rebalancing the churn variable produced much better results of the model from roc_auc value of 85% to 92%. hence a better classifying model.
We shall therefore consider is as our final classifier for the model.

In [ ]:
final_model2_GB=grid2_clf.best_estimator_

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(final_model2_GB, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
y_pred2=final_model2_GB.predict_proba(X_test)[:,1]#predict probabilities for positive class
#code below will threshold the probabilities to give class predictions
y_pred2 = np.where(y_pred2 > 0.5, 1,0)

In [ ]:
print(classification_report(y_test,y_pred2))

In [ ]:
#comfusion matrix plot
Label_1 = ['Predicted_Not_Churn', 'Predicted_Churn']
Label_2 = [ 'True Not_Churn','True Churn', ]
conf_matrix = confusion_matrix(y_test, y_pred2)
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, xticklabels=Label_1, yticklabels=Label_2, annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('')
plt.xlabel('Predicted class')
plt.show()

# CONCLUSION
# The resultant confusion matrix above show that there was an increase in the false positives as compared to the earlier confusion matrix from 96 to 164  and a decrease in the false negatvies from 178 to 153
# If the cost for the false positives is much high than the cost for false negatives, then more measures would be taken to mitigate possible higher customer churn.